In [ ]:
!pip install tensorflow numpy pandas matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

In [ ]:
np.random.seed(0)
t = np.linspace(0, 100, 500)
y = np.sin(t) + np.random.normal(0, 0.1, 500)

train_size = int(0.8 * len(t))
train_data, test_data = y[:train_size], y[train_size:]

# Define a function to create sequences for training
def create_sequences(data, seq_length):
    sequences = []
    target = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        target.append(data[i+seq_length])
    return np.array(sequences), np.array(target)

seq_length = 10
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [ ]:
# Reshape the input data for the CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Create a CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', 
input_shape=(seq_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))  # Output layer for regression

In [ ]:
# Compile the model
# Mean Squared Error for regression
model.compile(optimizer='adam', loss='mse')  

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, 
y_test), verbose=2)

# Make predictions
y_pred = model.predict(X_test)

In [ ]:
# Plot the original and predicted time series
plt.figure(figsize=(12, 6))
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, 
label='True')
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 
label='Predicted')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series Forecasting with CNN')
plt.show()